# Importación de librerías

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import re # Expresiones regulares
from sklearn import preprocessing # LabelEncoder

In [2]:
data = pd.read_table("../Dataset/labeledEligibilitySample1000000.csv", header = None)

C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
data.shape
data.sample(20)

,0,1
355269,__label__0,study interventions are Cisplatin . hereditary...
320828,__label__0,study interventions are Raloxifene Hydrochlori...
111723,__label__0,study interventions are Bevacizumab . recurren...
282365,__label__0,study interventions are Vaccines . stage iii a...
331794,__label__0,study interventions are Lenalidomide . ds stag...
446273,__label__0,study interventions are Hematoporphyrin Deriva...
761945,__label__1,study interventions are Bevacizumab . glioblas...
133872,__label__0,study interventions are Dexamethasone 21-phosp...
832828,__label__1,study interventions are Misonidazole . triple ...
287126,__label__0,study interventions are Immunoglobulins . recu...


In [4]:
clin_trial = pd.DataFrame(np.array(data).reshape(1000000,2), columns=['Label', 'Description'])
clin_trial[:].head(10)

,Label,Description
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...
5,__label__0,"study interventions are Antibodies, Monoclonal..."
6,__label__0,study interventions are Hormones . prostate ca...
7,__label__0,study interventions are Bendamustine Hydrochlo...
8,__label__0,study interventions are Nivolumab . recovered ...
9,__label__0,study interventions are Thalidomide . kidney c...


### Se transforma la variable elegible (la salida) en 0 y 1

In [5]:
clin_trial['Interventions'], clin_trial['Diagnoses'] = clin_trial['Description'].str.split('.', 1).str
clin_trial['Eligible'] = clin_trial['Label'].str.extract('(\d)', expand=True)

clin_trial=clin_trial.drop(['Description'], axis=1)
clin_trial=clin_trial.drop(['Label'], axis=1)

clin_trial.head(10)

,Interventions,Diagnoses,Eligible
0,study interventions are recombinant CD40-ligand,melanoma skin diagnosis and no active cns met...,0
1,study interventions are Liposomal doxorubicin,colorectal cancer diagnosis and cardiovascular,0
2,study interventions are BI 836909,multiple myeloma diagnosis and indwelling cen...,0
3,study interventions are Immunoglobulins,recurrent fallopian tube carcinoma diagnosis ...,0
4,study interventions are Paclitaxel,stage ovarian cancer diagnosis and patients m...,0
5,"study interventions are Antibodies, Monoclonal",recurrent verrucous carcinoma of the oral cav...,0
6,study interventions are Hormones,prostate cancer diagnosis and imaging examina...,0
7,study interventions are Bendamustine Hydrochlo...,diffuse large cell lymphoma diagnosis and no ...,0
8,study interventions are Nivolumab,recovered from all toxicities associated with...,0
9,study interventions are Thalidomide,kidney cancer diagnosis and no diabetes mellitus,0


In [6]:
# Se quita study interventions are puesto que está en todas las muestras de la varialbe Interventions
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace("study interventions are\s", "")
clin_trial.sample(10)

,Interventions,Diagnoses,Eligible
796267,Docetaxel,breast cancer diagnosis and patients with his...,1
724905,Idarubicin,multiple myeloma diagnosis and polyneuropathy...,1
872039,Doxorubicin,uncontrolled intercurrent illness including b...,1
680048,Cyclosporine,recurrent adult diffuse small cleaved cell ly...,1
100827,Prednisolone,prostate cancer diagnosis and has not receive...,0
979178,Rituximab,non hodgkin lymphoma nhl diagnosis and previo...,1
725261,Vidarabine,contiguous stage ii mantle cell lymphoma diag...,1
122388,Lenvatinib,stage iva thyroid gland follicular carcinoma ...,0
153978,Leucovorin,stage iv colon cancer diagnosis and no ongoin...,0
989675,Everolimus,recurrent pancreatic neuroendocrine carcinoma...,1


In [9]:
#Sirve para buscar palabras con esos caracteres

# for name in list(clin_trial.columns):
#     for sample in clin_trial[name]:
#         if re.findall('[+=*^#áÁéÉíÍóÓúÚ\[\]\$,.;:!@¡¿?\{\}\&`~\^\(\)|%_\-<>\'\"]', sample):
#             print(sample, "\n") ## Funciona para detectar caracteres especiales

In [10]:
# Se transforma todo a minúsculas
clin_trial['Interventions'] = clin_trial['Interventions'].str.lower() ##Todo a minúscula
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.lower()     ##Todo a minúscula

#Se reemplazan vocales con tildes en ambas variables
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[áäâà]', 'a', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[éêèë]', 'a', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[íïìî]', 'a', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[óôòö]', 'a', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[úûùü]', 'a', regex=True)

clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[áäâà]', 'a', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[éêèë]', 'a', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[íïìî]', 'a', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[óôòö]', 'a', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[úûùü]', 'a', regex=True)


clin_trial.sample(10)

,Interventions,Diagnoses,Eligible
169417,temozolomide,recurrent non small cell lung cancer diagnosi...,0
664954,cabozantinib,urinary bladder neoplasms diagnosis and infor...,1
88068,gemcitabine,metastatic pancreatic adenocarcinoma diagnosi...,0
438695,gene expression analysis,ovarian cancer diagnosis and see disease char...,0
527782,cypath®,have or have had cancer other than lung cancer,1
608100,jm 3100,extranodal marginal zone cell lymphoma of muc...,1
992066,dexamethasone acetate,plasma cell myeloma diagnosis and any serious...,1
597832,dynamic contrast-enhanced magnetic resonance i...,adult pineoblastoma diagnosis and renal impai...,1
240356,freund's adjuvant,intraocular melanoma diagnosis and renal,0
558980,interleukin-2,childhood solid neoplasm diagnosis and patien...,1


In [12]:
# Total de categorías diferentes en la variable Interventions
len(np.unique(clin_trial['Interventions'])) 

14363

In [19]:
# Sirve para ver cuantos datos hay por categoría de interventions

datosInt = clin_trial['Interventions'].value_counts()
datosInt

# c = 0
# for date in datosInt:
#     if date > 0:
#         c +=1
        
# print(c)

1068     34457
1065     22897
9405     22732
811      21169
1628     20445
3472     19992
6462     19829
2271     19210
2655     16940
5137     15064
5138     14654
5485     14555
11913    13689
3983     13490
4740     13094
7448     12589
3931     12462
7155     12397
13778    11186
8516     10856
11296    10756
8517     10580
5178     10099
13898     9815
9372      9777
2233      9464
6963      9132
4687      8988
13026     8242
4614      7951
         ...  
14134        1
813          1
13099        1
5928         1
300          1
3883         1
4707         1
8495         1
4905         1
2858         1
8293         1
11054        1
9007         1
11566        1
10031        1
8488         1
4398         1
4399         1
7779         1
5730         1
9000         1
1632         1
6959         1
9512         1
9316         1
3169         1
7983         1
4706         1
10536        1
0            1
Name: Interventions, Length: 14363, dtype: int64

In [20]:
# Total de categorías diferentes en la variable Diagnoses
len(np.unique(clin_trial['Diagnoses']))

638745

In [18]:
datosDiag = clin_trial['Diagnoses'].value_counts() 
datosDiag

 lymphoma diagnosis and not specified                                                                                                                                                                                                                                                                                                                                                                                                                                                       812
 breast cancer diagnosis and not specified                                                                                                                                                                                                                                                                                                                                                                                                                                                  558
 lymphoma diagnosis and disease characte

## Codificación de la variable Interventions

In [15]:
labels = np.unique(clin_trial['Interventions'])

lb_interventions = preprocessing.LabelEncoder()
lb_interventions.fit(labels)
clin_trial['Interventions'] = lb_interventions.transform(clin_trial['Interventions']) 

In [16]:
clin_trial

,Interventions,Diagnoses,Eligible
0,11034,melanoma skin diagnosis and no active cns met...,0
1,7448,colorectal cancer diagnosis and cardiovascular,0
2,1655,multiple myeloma diagnosis and indwelling cen...,0
3,6462,recurrent fallopian tube carcinoma diagnosis ...,0
4,9405,stage ovarian cancer diagnosis and patients m...,0
5,1068,recurrent verrucous carcinoma of the oral cav...,0
6,6114,prostate cancer diagnosis and imaging examina...,0
7,1593,diffuse large cell lymphoma diagnosis and no ...,0
8,8799,recovered from all toxicities associated with...,0
9,13026,kidney cancer diagnosis and no diabetes mellitus,0


# Remover Stop Words

## Preliminares

In [2]:
# Se carga la libreria
from nltk.corpus import stopwords

# Se debe descargar el conjunto de 'Stop Words' la primera vez
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/juan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Crear las palabras Token

In [3]:
# Subo este ejemplo pero aquí entrarían los registros una vez tokenizados
tokenized_words = ['i', 'am', 'going', 'to', 'go', 'to', 'the', 'store', 'and', 'park']

## Cargar las Stop Words

In [4]:
# Carga las Stop Words en inglés
stop_words = stopwords.words('english')

## Estaría bien revisar este conjunto 'english'. Pero supongo que es el más útil para nuestro caso también.
# Imprime algunas Stop Words
stop_words[:100]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once']

## Remover las Stop Words

In [5]:
# Remueve las Stop Words del ejemplo que hay más arriba. Cambiarlo una vez se tengan los registros tokenizados
[word for word in tokenized_words if word not in stop_words]

['going', 'go', 'store', 'park']